 # Mental Representations of Psychological Disorders: A Reverse Correlation Approach

 ## Abstract

- Mental health stigma remains a pervasive societal issue, influencing interpersonal behavior, access to care, and policy support. While explicit attitudes have been widely studied, less is known about how individuals visually conceptualize mental illness in others. This study investigates the mental representations people hold of various psychological disorders by leveraging a reverse correlation approach to face perception.

- Participants completed a categorization task where they judged AI-generated faces for resemblance to individuals with specific mental health conditions (depression, anxiety, bipolar disorder, and PTSD). The images were randomly sampled from a generative adversarial network trained on neutral human faces, allowing systematic mapping of participants visual intuitions. From these responses, individual-level visual prototypes were estimated.

- These prototypes were analyzed using computational models of social perception to assess how visual representations align with attributes like trustworthiness, dominance, and attractiveness. The strength and valence of these impressions were related to participants self-reported stigma toward mental illness.

- Results demonstrate that visual representations of mental illness systematically differ from non-illness categories along socially relevant dimensions, and these differences scale with individual variation in stigma. By making implicit visual stereotypes explicit, this study offers a novel framework for quantifying how social bias becomes encoded in mental imagery.


## Introduction

### Background and Significance

Mental health stigma constitutes a significant barrier to treatment seeking and social integration for individuals with psychological disorders. While substantial research has examined explicit stereotypes through self-report measures, the visual components of these stereotypes remain understudied. The human brain relies heavily on facial cues when forming social judgments (Todorov et al., 2015), making visual representations a critical yet overlooked aspect of mental illness stigma.

Reverse correlation techniques have emerged as a powerful tool for revealing the mental templates underlying social judgments. This method involves having participants make judgments about randomly varying stimuli, then statistically reconstructing the implicit template guiding those judgments. Applied to face perception, this approach can reveal how people visually conceptualize social categories - including stigmatized groups.

### Current Study

This investigation employs a novel combination of generative adversarial networks (GANs) and reverse correlation to:
1. Map visual prototypes for four psychological disorders (MDD, GAD, BPD, PTSD)
2. Quantify how these representations differ from neutral faces
3. Examine relationships between prototype characteristics and explicit stigma measures

The study advances stigma research by moving beyond verbal reports to examine the visual basis of stereotyping, with implications for anti-stigma interventions.



## Methods

### Participants

A total of N=300 participants will be recruited through university subject pools. Participants completed:
- Reverse correlation face judgment task

- For each image, participants must select one of three options:
  - `{X}` (e.g., a specific judgment assigned to the participant)
  - `not {X}`
  - `not sure`
- The judgment `{X}` is assigned to each participant based on the experimental cond

- Demographic questionnaire
- Mental Illness Stigma Scale (MISS; α=.89)
- Debriefingi
tion.
Sample size was determined through power analysis for planned within-subject comparisons of face prototypes.

### Stimuli

Faces were generated using StyleGAN2 (Karras et al., 2020) trained on the FFHQ dataset. From the latent space, we sampled 300 base faces meeting criteria for:
- Neutral emotional expression
- Balanced gender presentation
- Diverse racial/ethnic appea
- The aim is to recruit **30 participants per experimental condition**.
- The total target sample size is **120-150 participants across all conditions**.rance

Each face was rendered at 512×512 resolution with consistent lighting and background.



### Procedure

The experiment consisted of three phases:

1. **Training Phase (10 trials)**
   - Familiarized participants with task requirements
   - Provided examples of "prototypical" faces (not used in main task)

2. **Main Task (300 trials)**
   - Each trial presented one randomly generated face
   - Participants judged whether face resembled someone with:
     - Their assigned condition (e.g., "PTSD")
     - "Not [condition]"
     - "Not sure"
   - 10% of trials were repeats for reliability assessment

3. **Survey Measures**
   - Demographic questionnaire
   - Stigma scale (MISS)
   - Debriefing questions about task strategies

Total session duration averaged 45 minutes. Participants received course credit or $15 compensation.


In [ ]:
%reset -f
%matplotlib inline
import os, sys, subprocess
import ctypes



# Ensure that it can see `fused.so`, `libcudart.so.10.0`, `libcudnn_cnn_infer.so.8` and `libcuda.so`
# Also make sure to update your .bashrc LD_LIBRARY_PATH to include the nonsense below.
my_env = os.environ.copy()
my_home_dir = "/home/stefanu/"
paths_to_include = [
    f"{my_home_dir}envs/pytorch/lib/python3.10/site-packages/nvidia/cudnn/lib",
    f"{my_home_dir}envs/pytorch/lib/python3.10/site-packages/torch/lib",
    f"{my_home_dir}envs/pytorch/bin",
    f"{my_home_dir}envs/pytorch/lib",
    f"{my_home_dir}envs/pytorch/targets/x86_64-linux/include",
    f"{my_home_dir}envs/pytorch/lib/stubs",
]
paths_to_include_string = ":".join(paths_to_include)
my_env["PATH"] = f"{paths_to_include_string}:" + my_env["PATH"]
my_env["CPATH"] = f"{my_home_dir}envs/pytorch/lib/python3.10/site-packages/nvidia/cudnn/lib:{my_home_dir}envs/pytorch/targets/x86_64-linux/include:{my_home_dir}envs/pytorch-kai/lib/python3.10/site-packages/torch/lib"
my_env["LD_LIBRARY_PATH"] = f"{my_home_dir}envs/pytorch/lib/python3.10/site-packages/nvidia/cudnn/lib:{my_home_dir}envs/pytorch-kai/lib/stubs:{my_home_dir}envs/pytorch-kai/lib/python3.10/site-packages/torch/lib:{my_home_dir}envs/pytorch-kai/targets/x86_64-linux/lib:{my_env.get('LD_LIBRARY_PATH')}"

os.environ.update(my_env)

ctypes.CDLL(f"{my_home_dir}envs/pytorch/lib/python3.10/site-packages/nvidia/cudnn/lib/libcudnn_cnn_infer.so.8", mode=ctypes.RTLD_GLOBAL)
# ctypes.CDLL(f'{my_home_dir}envs/pytorch-kai/lib/python3.10/site-packages/torch/lib/libcudart.so.10.0', mode=ctypes.RTLD_GLOBAL) # file doesn't exist

from ipywidgets import interact
from pathlib import Path
from tqdm import tqdm


import numpy as np
from PIL import Image
import torch
import pickle

import pandas as pd
import json
from IPython.display import display
import os
from collections import defaultdict


import pytz
import janitor
import scipy.stats as stats
from scipy.stats import pearsonr
import random
import pingouin as pg

In [7]:
main_dir = Path(f"{my_home_dir}repos/modeling-tools/")
ckpt = main_dir / "pretrained" / "NAMFHQ-config-f-004000.pt"

In [8]:
import sys
print(sys.executable)

/home/cy36/envs/pytorch/bin/python


In [9]:
sys.path.append(".")
sys.path.append("..")

from Generators import *
from Projectors import *

from Config import BaseGeneratorOpts, BaseProjectorOpts, FeatureStyleEncoderOpts, Pixel2Style2PixelOpts
from EncoderDecoder import EncoderDecoder
from Models import MultiAttributeModel
from utils.common import tensor2im, concat_imgs, convert_tensor_to_numpy

/home/cy36/envs/pytorch/lib/python3.10/site-packages/torch/utils/cpp_extension.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import packaging  # type: ignore[attr-defined]


StyleGAN2: Optimized CUDA op FusedLeakyReLU available. Compiling...
StyleGAN2: Optimized CUDA op UpFirDn2d available. Compiling...


In [10]:
base_generator = StyleGAN2(BaseGeneratorOpts(ckpt=ckpt))

In [11]:
base_projector = BaseProjector(
    generator=base_generator.generator,
    projector_opts=BaseProjectorOpts(
        ckpt=ckpt,
        step=1000,
    ),
)

No align_ckpt specified, using default:  ./pretrained/shape_predictor_68_face_landmarks.dat


In [12]:
ed = EncoderDecoder(generator=base_generator, projector=base_projector)

In [ ]:
model = MultiAttributeModel()
model.load("/home/stefanu/repos/modeling-tools/models/2024-01-29_omnibus_model.p")

In [ ]:
utc = pytz.UTC

# ROOT_PATH = Path("/projects/illinois/las/psych/stefanu/projects/reverse-correlation/helper-ses-mindworks/")
ROOT_PATH = Path("/home/stefanu/repos/modeling-tools/output")
DATE_PATH = Path("2025-04-29/")
DATA_PATH = Path("/home/stefanu/repos/modeling-tools/notebooks")  # data from jspsych
DATA_FILE = "Data/jspsych_data.csv"
LATENT_PATH = ROOT_PATH / "Latents"  # path to dlatents of rc images
SAVE_PATH = (
    ROOT_PATH / DATE_PATH / "results"
)  # where you want to save the images/arrays/latents; each subject gets their own folder
LEVELS = ["negative", "positive"]
CONDITION_DICT = {
    "mdd": "MDD",
    "bpd": "BPD", 
    "gad": "GAD",
    "ptsd": "PTSD"
}
CONDITIONS = list(CONDITION_DICT.values())
CATEGORY_LABEL_DICT = {
    "positive": "yes",
    "negative": "no",
    "neither": "not sure",
}

CONDITION_MAP = {
    'gad': 'GAD',
    'mdd': 'MDD', 
    'bpd': 'BPD',
    'ptsd': 'PTSD'
}


FIGURE_PATH = SAVE_PATH / "figures"
SEED = 628884

IDS_TO_EXCLUDE = [
    "XXX",
    "95165"
]
modeling_tools_dir = "../repos/modeling-tools/"
modeling_tools_dir = Path(modeling_tools_dir)
Path(SAVE_PATH).mkdir(exist_ok=True, parents=True)

### Data Processing Pipeline

The analysis involved multiple stages of data transformation and quality control:


#### 1. Initial Data Loading and Cleaning


In [ ]:
def load_data(data_dir=DATA_PATH, data_file=DATA_FILE):
    return pd.read_csv(data_dir / data_file).clean_names(case_type="snake")


def load_rc_latents(path=LATENT_PATH):
    return np.load(path / "latents.npz")["data"]


def get_concat_h_multi_resize(im_list, resample=Image.BICUBIC):
    min_height = min(im.height for im in im_list)
    im_list_resize = [
        
        im.resize(
            (int(im.width * min_height / im.height), min_height), resample=resample
        )
        for im in im_list
    ]
    total_width = sum(im.width for im in im_list_resize)
    dst = Image.new("RGB", (total_width, min_height))
    pos_x = 0
    for im in im_list_resize:
        dst.paste(im, (pos_x, 0))
        pos_x += im.width
    return dst

#### 2. Data Quality Control Functions

In [ ]:
def broadcast_screen_dimensions(df):
    """
    Fill in missing screen_width and screen_height values for each worker_id
    by copying values from the first row where they appear.
    
    Args:
        df (pd.DataFrame): DataFrame with worker_id, screen_width, screen_height columns,
                          where dimensions are only populated in the first row per worker.
                          
    Returns:
        pd.DataFrame: DataFrame with screen dimensions filled for all rows per worker.
    """
    if not all(col in df.columns for col in ["worker_id", "screen_width", "screen_height"]):
        print("Warning: Missing required columns. Cannot broadcast screen dimensions.")
        return df
    
    # Create a copy to avoid modifying the original
    result_df = df.copy()
    
    # Use first_valid_index and transform
    for dim_col in ["screen_width", "screen_height"]:
        # Get first non-null value for each worker_id
        first_values = df.groupby("worker_id")[dim_col].first()
        # Map those values back to all rows for that worker
        result_df[dim_col] = result_df["worker_id"].map(first_values)
    
    return result_df


def calculate_reliability(df, expected_pairs=30):
    """
    
    Calculates test-retest reliability for each worker based on repeated stimuli,

    checking for the expected number of pairs.
    
    Args:
        df (pd.DataFrame): DataFrame containing experiment data with columns

        'worker_id', 'stimulus_number', 'repeat', and 'response_label'.

        expected_pairs (int): The expected number of repeated stimuli pairs per worker.
    
    Returns:
       pd.Series: Series mapping worker_id to their test-retest Pearson correlation.
        
       Returns np.nan for workers who don't have exactly expected_pairs,

       have zero variance in responses, or other calculation issues.
    """
    # Filter to only reverse correlation trials
    rc_data = df[df['trial_type'] == 'single-stim-rev-cor-trial'].copy()
    
    # Clean repeat column
    rc_data['repeat'] = rc_data['repeat'].replace(
    {'True': True, 'False': False, 'true': True, 'false': False}
    ).astype(bool)
    
    # Comprehensive response mapping
    response_map = {
        "GAD": 1, "no GAD": -1, 
        "MDD": 1, "no MDD": -1,
        "PTSD": 1, "no PTSD": -1, 
        "BPD": 1, "no BPD": -1,
        "yes": 1, "no": -1, "not sure": 0
    }
    rc_data['score'] = rc_data['response_label'].map(response_map)
    rc_data = rc_data.dropna(subset=['score'])
    
    # Group by participant
    reliability_scores = {}
    
    for worker_id, worker_data in rc_data.groupby('worker_id'):
        # Get all stimuli with exactly 2 presentations (1 first, 1 repeat)
        stimulus_counts = worker_data['stimulus_number'].value_counts()
        paired_stimuli = stimulus_counts[stimulus_counts == 2].index
        
        # Verify proper pairing (exactly 1 first and 1 repeat presentation)
        valid_pairs = []
        for stim in paired_stimuli:
            stim_data = worker_data[worker_data['stimulus_number'] == stim]
            first = stim_data[~stim_data['repeat']]
            repeat = stim_data[stim_data['repeat']]
            
            if len(first) == 1 and len(repeat) == 1:
                valid_pairs.append((
                    first.iloc[0]['score'],
                    repeat.iloc[0]['score']
                ))
        
        # Calculate reliability if we have sufficient pairs
        if len(valid_pairs) >= 2:
            try:
                first_resp, repeat_resp = zip(*valid_pairs)
                corr = pearsonr(first_resp, repeat_resp)[0]
                
                # Check if we got the expected number of pairs
                if len(valid_pairs) != expected_pairs:
                    print(f"Warning: {worker_id} has {len(valid_pairs)} pairs (expected {expected_pairs})")
                
                reliability_scores[worker_id] = corr
            except:
                reliability_scores[worker_id] = np.nan
        else:
            reliability_scores[worker_id] = np.nan
    
    return pd.Series(reliability_scores, name='pearson_r')





def get_main_data(data, include_repeat_data=True):
    """
    Processes experiment data and returns the main experiment data.



    Args:

        data (pd.DataFrame): Raw DataFrame from experiment data.

        include_repeat_data (bool): If True, includes repeat trials for reliable

                                     participants in the output 'main_data'.

                                     If False, removes repeat trials.

        expected_repeat_pairs (int): Expected number of stimulus pairs for reliability check.





    Returns:

        pd.DataFrame: Filtered and cleaned DataFrame for all participants.
    """
    # Filter by experiment phase
    valid_phases = ["main", "main_repeat"]
    main_data = data.loc[data["experiment_phase"].isin(valid_phases)].copy()
    
    if main_data.empty:
        print("Warning: No data found for experiment_phase == 'main'.")
        return pd.DataFrame()  
    
    # Convert times
    try:
        main_data["start_time"] = pd.to_datetime(main_data["start_time"])
        main_data["end_time"] = pd.to_datetime(main_data["end_time"])
    except Exception as e:
        print(f"Warning: Could not convert time columns: {e}")
    
    # Get first session per worker
    if "start_time" in main_data.columns and "anon_id" in main_data.columns:
        min_start_times = main_data.groupby("anon_id")["start_time"].min()
        main_data = main_data.merge(min_start_times.rename("min_start_time"), 
                                  on="anon_id", how="left")
        main_data = main_data.loc[main_data["start_time"] == main_data["min_start_time"]]
        main_data = main_data.drop(columns=["min_start_time"])
    
    # Apply repeat data filtering
    if not include_repeat_data:
        main_data = main_data.loc[~main_data["repeat"]]
    
    # Clean stimulus info
    if "stimulus" in main_data.columns:
        main_data["stimulus"] = main_data["stimulus"].str.replace("src/images/main/", "", regex=False)
        main_data["latent"] = main_data["stimulus"].str.replace(".jpg", ".npy", regex=False)
        main_data["stimulus_index"] = pd.to_numeric(
            main_data["stimulus"].str.replace(".jpg", "", regex=False), 
            errors="coerce"
        )
    
    return main_data  


def clean_data(data, ids_to_exclude=IDS_TO_EXCLUDE):
    return data.loc[~data["worker_id"].isin(ids_to_exclude)]


def get_repeat_data(main_data):
    return main_data.loc[main_data["repeat"] == True]


def get_simplified_race(races):
    if len(races) == 1:
        return races[0]
    return "Two or more races"


def are_all_elements_integers(lst):
    return all(isinstance(item, int) for item in lst)


def dict_to_list(dict_input):
    max_index = max(int(key) for key in dict_input.keys())
    sorted_items = sorted(dict_input.items(), key=lambda x: int(x[0]))
    result_list = [None] * (max_index + 1)

    for key, value in sorted_items:
        result_list[int(key)] = value

    if not are_all_elements_integers(result_list):
        raise ValueError(f"Not all elements are integers: {result_list}")

    return result_list


def get_meta_data(
    data,
    survey_types=["demographic_survey", "debriefing_survey"],
    worker_id_col="worker_id",
    expected_repeat_pairs=30,
    reliability_must_be_above=0,
    seriousness_threshold=70,
    min_pixel_count=480_000,
):
    """
     Processes experiment data and returns the main experiment data.



    Args:

        data (pd.DataFrame): Raw DataFrame from experiment data.

        include_repeat_data (bool): If True, includes repeat trials for reliable

                                     participants in the output 'main_data'.

                                     If False, removes repeat trials.

        expected_repeat_pairs (int): Expected number of stimulus pairs for reliability check.





    Returns:

        pd.DataFrame: Filtered and cleaned DataFrame for all participants.


    """
 
    id_columns = ['anon_id', 'worker_id', 'sona_id']
    available_ids = [col for col in id_columns if col in data.columns]
    if not available_ids:
        raise ValueError("No valid ID columns found in data")
    primary_id = available_ids[0]
    

    trial_phases = ["main", "main_repeat"]
    main_data = data.loc[data["experiment_phase"].isin(trial_phases)].copy()
    

    if "start_time" in main_data.columns:
        if not pd.api.types.is_datetime64_any_dtype(main_data["start_time"]):
            main_data["start_time"] = pd.to_datetime(main_data["start_time"], format='ISO8601')
        min_start_times = main_data.groupby(primary_id)["start_time"].min()
        main_data = main_data.merge(min_start_times.rename("min_start_time"), on=primary_id, how='left')
        df_first_session = main_data.loc[main_data["start_time"] == main_data["min_start_time"]].copy()
        df_first_session = df_first_session.drop(columns=["min_start_time"])
    else:
        print("No start_time")
        df_first_session = main_data
    

    reliability_scores = calculate_reliability(df_first_session, expected_pairs=expected_repeat_pairs)

 
    response_counts_by_subject = main_data.groupby(by=primary_id)["response_label"].value_counts()
    reaction_times = main_data.groupby(by=[primary_id, "response_label"])["rt"]
    reaction_times_by_subject_and_response = reaction_times.mean()
    reaction_times_by_subject_and_response_stds = reaction_times.std()


    id_to_condition = main_data.groupby(by=primary_id)["condition"].first()
    

    screen_dimensions = data.groupby(primary_id)[["screen_width", "screen_height"]].first()
    

    grouped = data.groupby(by=primary_id)
    rows = []
    
    for subject_id, group in grouped:
        survey_dataframes = []
        
        for survey_type in survey_types:
            survey_data = (
                group.query(f'experiment_phase == "{survey_type}"')
                .dropna(subset=["form_data"])
                .loc[:, [primary_id, "form_data"]]
            )
            
            if survey_data.empty:
                continue
                
            try:
                survey_data.form_data = survey_data.form_data.apply(json.loads).apply(
                    lambda x: pd.DataFrame([x])
                )
                survey_df = pd.concat([f for _, f in survey_data.form_data.items()])
                survey_dataframes.append(survey_df)
            except Exception as e:
                print(f"Error processing survey data for {primary_id} {subject_id}, survey {survey_type}: {e}")
                continue
        

        row = {
            primary_id: subject_id,
            "condition": id_to_condition.get(subject_id, None),
            **{col: group[col].iloc[0] for col in id_columns if col in group.columns}
        }
        

        response_counts = response_counts_by_subject.get(subject_id, {})
        for resp in ["yes", "no", "not sure"]:
            row[f"{resp}_count"] = response_counts.get(resp, 0)
            row[f"{resp}_rt_mean"] = reaction_times_by_subject_and_response.get((subject_id, resp), np.nan)
            row[f"{resp}_rt_sd"] = reaction_times_by_subject_and_response_stds.get((subject_id, resp), np.nan)
        

        if survey_dataframes:
            combined_data = pd.concat(survey_dataframes, axis=1)
            collapsed_data = combined_data.apply(
                lambda x: x.dropna().iloc[0] if not x.dropna().empty else None)
            row.update(collapsed_data.to_dict())
        
        # Add screen dimensions
        if subject_id in screen_dimensions.index:
            row.update({
                "screen_width": screen_dimensions.loc[subject_id, "screen_width"],
                "screen_height": screen_dimensions.loc[subject_id, "screen_height"],
                "screen_area": screen_dimensions.loc[subject_id, "screen_width"] * 
                              screen_dimensions.loc[subject_id, "screen_height"]
            })
        
        # Add reliability score
        if reliability_scores is not None and subject_id in reliability_scores.index:
            row["reliability_score"] = reliability_scores.loc[subject_id]
        
        # Set exclusion criteria
        exclusion_flags = {
            "seriousness_low": "seriousness" in row and row["seriousness"] < seriousness_threshold,
            "screen_size_low": "screen_area" in row and row["screen_area"] < min_pixel_count,
            "interrupted_survey": "interruption" in row and row["interruption"] and "yes" in str(row["interruption"]).lower(),
            "previously_participated": "participatedBefore" in row and row["participatedBefore"] and 
                                     "yes" in str(row["participatedBefore"]).lower(),
            "reliability_low": "reliability_score" in row and 
                              (row["reliability_score"] <= reliability_must_be_above or 
                               np.isnan(row["reliability_score"]))
        }
        row.update(exclusion_flags)
        row["is_bad_subject"] = any(exclusion_flags.values())
        
        rows.append(row)
    
    meta_data = pd.DataFrame.from_records(rows)
    

    if hasattr(meta_data, 'clean_names'):
        meta_data = meta_data.clean_names(case_type="snake")
    

    if "race" in meta_data.columns:
        meta_data["simplified_race"] = meta_data["race"].apply(get_simplified_race)
    # meta_data = meta_data[meta_data['condition'].isin(CONDITION_MAP.values())]
    return meta_data

def get_category_latents(
    subject_data,
    category,
    latents,
    category_col="response_label",
    latent_col="stimulus_index",
):
    indexes = subject_data.loc[subject_data[category_col] == category][
        latent_col
    ].tolist()
    return latents[indexes, :]



def get_results(
    subject_data,
    # survey_data, # XXX come back to survey data stuff
    latents,
    positive_category=CATEGORY_LABEL_DICT["positive"],
    negative_category=CATEGORY_LABEL_DICT["negative"],
    neither_category=CATEGORY_LABEL_DICT["neither"],
    desired_num_trials=5,
    num_response_options=3,
    num_random_latents=30,
    min_sd=-1.5,  # -8
    max_sd=1.5,  # 8
    step=0.5,  # 1
    save_path=SAVE_PATH,
    save_output=False,
    seed=SEED,
):
    random.seed(seed)

    subject_id = subject_data["anon_id"].unique()[0]
    print(f"subject = {subject_id}")
    # survey_data = survey_data.loc[survey_data["anon_id"] == subject_id] # xxx
    # survey_data_dict = survey_data.to_dict(orient="records")[0] # xxx
    conditions = subject_data["condition"].unique()
    print("conditions:",conditions)
    # instruction_conditions = subject_data["instructions_condition"].unique()
    # scenarios = subject_data["scenario"].unique()
    if len(conditions) != 1:
        raise ValueError("There should be exactly one condition per subject!")

    condition = conditions[0]
    # instruction_condition = instruction_conditions[0]
    # scenario = scenarios[0]

    subject_save_path = save_path / condition / subject_id
    reel_save_path = save_path / condition / "reels"

    if not subject_save_path.exists():
        subject_save_path.mkdir(parents=True)

    if not reel_save_path.exists():
        reel_save_path.mkdir(parents=True)

    response_label_dict = subject_data.groupby("response_label").size()

    if response_label_dict.sum() < desired_num_trials:
        raise ValueError(f"Less than {desired_num_trials} trials!")

    has_no_neither = neither_category not in response_label_dict.keys()

    if response_label_dict.shape[0] < num_response_options and not has_no_neither:
        raise ValueError(
            f"Need at least one response per (unreplaceable) option! Subject: {subject_id}"
        )

    neither_latents = None
    possible_latent_indexes = subject_data["stimulus_index"].unique().tolist()

    if has_no_neither:
        print(f"Replacing '{neither_category}' with random average...")
        neither_latent_indexes = random.sample(
            possible_latent_indexes, num_random_latents
        )
        neither_latents = latents[neither_latent_indexes, :]
    else:
        neither_latents = get_category_latents(
            subject_data, category=neither_category, latents=latents
        )

        if (
            neither_latents.shape[0] < num_random_latents
            and neither_latents.shape[0] >= 1
        ):
            print(
                f"{subject_id}: only {neither_latents.shape[0]} responses in '{neither_category}' category; adding more to reach {num_random_latents}..."
            )


            neither_latent_indexes = subject_data.loc[
                subject_data["response_label"] == neither_category
            ]["stimulus_index"].tolist()
            current_num_indexes = len(neither_latent_indexes)
            possible_latent_indexes = subject_data["stimulus_index"].unique().tolist()
            remaining_possible_indexes = set(possible_latent_indexes) - set(
                neither_latent_indexes
            )
            additional_neither_latent_indexes = random.sample(
                list(remaining_possible_indexes),
                num_random_latents - current_num_indexes,
            )
            if (
                len(neither_latent_indexes)
                + len(additional_neither_latent_indexes)
                != num_random_latents
            ):
                raise ValueError(
                    f"Error at {subject_id}: originally {len(neither_latent_indexes)} responses, adding {len(additional_neither_latent_indexes)} != {num_random_latents} !"
                )

            new_neither_latent_indexes = [
                *neither_latent_indexes,
                *additional_neither_latent_indexes,
            ]
            neither_latents = latents[new_neither_latent_indexes, :]

    positive_latents = get_category_latents(
        subject_data, category=positive_category, latents=latents
    )
    negative_latents = get_category_latents(
        subject_data, category=negative_category, latents=latents
    )
    all_latents = [*positive_latents, *negative_latents, *neither_latents]
    print(f"Positive latents: {len(positive_latents)}")
    print(f"Negative latents: {len(negative_latents)}") 
    print(f"Neither latents: {len(neither_latents)}")
    positive_mean = np.mean(np.stack(positive_latents), axis=0)
    negative_mean = np.mean(np.stack(negative_latents), axis=0)
    neither_mean = np.mean(np.stack(neither_latents), axis=0)
    all_mean = np.mean(np.stack(all_latents), axis=0)
    # XXX Come back to whether we should just look at the direction for correlations
    # tmr_vector = positive_mean - negative_mean + neither_mean
    tmr_vector = positive_mean - negative_mean # just the direction

    images = []
    result_latents = []
    deepface_analyses = []
    our_model_analyses = []
    model_correlations = []
    try:
        for s in np.arange(min_sd, max_sd + step, step):

            result = create_mental_representations(
                encoder_decoder=ed,
                positive=positive_mean,
                negative=negative_mean,
                neutral=neither_mean,
                step_num=s,
                norm=True,
                mixed_norm=False,
                eps=1e-8,
                idio=False,
            )

            our_model_analysis = model.predict_all(result["latents"])
            # deepface_analysis = DeepFace.analyze(
            #     img_path=np.array(result["image"]),
            #     actions=["age", "gender", "race", "emotion"],
            #     prog_bar=False,
            # )
            # deepface_analyses.append(deepface_analysis)
            images.append(result["image"])
            result_latents.append(result["latents"])
            our_model_analyses.append(our_model_analysis)

            # except:
            #     display(result["image"])
            #     print(f"{s}: error")

            if save_output:
                result["image"].save(
                    subject_save_path / f"{subject_id}_{condition}_{s}.jpg"
                )
                np.save(
                    subject_save_path / f"{subject_id}_{condition}_{s}.npy",
                    result["latents"],
                )

        reel = get_concat_h_multi_resize(images)

        if save_output:
            reel.save(reel_save_path / f"{subject_id}_{condition}_reel.jpg")

        # first_analysis = deepface_analyses[0]
        # last_analysis = deepface_analyses[-1]

        first_face_our_model_analysis = our_model_analyses[0]
        first_face_our_model_analysis = {
            f"negative_{key}": value
            for key, value in fir
            
        st_face_our_model_analysis.items()
        }
        last_face_our_model_analysis = our_model_analyses[-1]
        last_face_our_model_analysis = {
            f"positive_{key}": value
            for key, value in last_face_our_model_analysis.items()
        }

        attributes = list(model.factors.keys())
        for attribute in attributes:
            vector = model.get_factor(attribute)["coefficients"]
            this_corr = pg.corr(vector, tmr_vector)
            this_corr = this_corr.rename(
                {
                    "r": f"tmr_r_{attribute}",
                    "p-val": f"tmr_r_pval_{attribute}",
                    "CI95%": f"tmr_r_CI95%_{attribute}",
                    "BF10": f"tmr_r_BF10_{attribute}",
                    "power": f"tmr_r_power_{attribute}",
                    "n": f"tmr_r_n_{attribute}",
                },
                axis=1,
            )

            model_correlations.append(this_corr.to_dict("records")[0])

        model_correlations = {k: v for d in model_correlations for k, v in d.items()}

        return {
            # **survey_data_dict, # xxx
            "condition": condition,
            # "scenario": scenario,
            # "instructions_condition": instruction_condition,
            "positive": positive_latents,
            "negative": negative_latents,
            "neither": neither_latents,
            "positive_mean": positive_mean,
            "negative_mean": negative_mean,
            "neither_mean": neither_mean,
            "tmr_vector": tmr_vector,
            "all": all_latents,
            # "images": images, # XXX
            # "reel": reel,
            "our_model_analyses": our_model_analyses,
            **first_face_our_model_analysis,
            **last_face_our_model_analysis,
            **model_correlations,
            # "deepface_analyses": deepface_analyses,
            # "negative_age": first_analysis["age"],
            # "negative_gender": first_analysis["gender"],
            # "negative_dominant_race": first_analysis["dominant_race"],
            # "negative_dominant_emotion": first_analysis["dominant_emotion"],
            # "positive_age": last_analysis["age"],
            # "positive_gender": last_analysis["gender"],
            # "positive_dominant_race": last_analysis["dominant_race"],
            # "positive_dominant_emotion": last_analysis["dominant_emotion"],
        }
    except Exception as e:
        print(f"error in subject: {subject_id}: {e}")


def get_condition_specific_labels(condition):
    """Returns response labels dynamically based on condition."""
    condition = condition.upper()  # Ensure "gad" -> "GAD"
    return {
        "positive": condition,           # e.g., "GAD"
        "negative": f"no {condition}",   # e.g., "no GAD"
        "neither": "not sure"
    }

### Prototype Generation

For each participant, we computed their Target Mental Representation (TMR) using:

 TMR = X - N + U 
  where X= average of images selected for yes, N= average of images selected as no, U = average of images selected as not sure.


In [ ]:
def create_mental_representations(
    encoder_decoder,
    positive,
    negative,
    neutral,
    step_num,
    norm=True,
    mixed_norm=False,
    eps=1e-8,
    idio=False
):
    """
    Create mental representations based on the given vectors.

    Args:
        encoder_decoder: The encoder-decoder model.
        positive: A vector that represents the high end of the scale (or target judgment).
        negative: A vector that represents the low end of the scale (or anti-target judgment).
        neutral: A vector that the unique values are applied (either mean of selected "neutrals" or mean of a random selection).
        step_num: An integer to multiply the vector values by (if normlized with `norm=True`, this can be interpreted as +/-SDs).
        norm: A boolean indicating whether to normalize the vectors (default: True).
        mixed_norm: A boolean indicating whether to use mixed normalization whereby the "neutral" vector is left unnormalized (default: False).
        eps: A small value to prevent division by zero (default: 1e-8).
        idio: A boolean indicating whether to return the idiosyncratic model vector (default: False).

    Returns:
        A tuple containing the output image, the decoded tensor, and the positive-negative vector (if idio=True).
    """
    if norm:
        # Get neutral magnitude for later unnormalization
        neutral_magnitude = np.linalg.norm(neutral)

        # Normalize both vectors
        pos_neg = positive - negative
        diff_norm = np.linalg.norm(pos_neg) + eps
        normalized_diff = pos_neg / diff_norm

        normalized_neutral = neutral / (np.linalg.norm(neutral) + eps)

        # Combine normalized vectors
        combined = normalized_neutral + (step_num * normalized_diff)

        pos_neg_out = combined * neutral_magnitude

    elif mixed_norm:
        # pos_neg = (positive - negative) / np.linalg.norm(positive - negative)
        # pos_neg_out = (pos_neg * step_num) + neutral

        # Normalize only the difference vector
        pos_neg = positive - negative
        diff_norm = np.linalg.norm(pos_neg) + eps
        normalized_diff = pos_neg / diff_norm

        pos_neg_out = (step_num * normalized_diff) + neutral

    else:
        pos_neg = positive - negative
        pos_neg_out = (pos_neg * step_num) + neutral

    to_decode = torch.from_numpy(pos_neg_out).cuda().float()

    with torch.no_grad():
        out = encoder_decoder.decode(to_decode)

    if idio:
        return {
            "image": tensor2im(out.squeeze()), 
            "latents": to_decode.cpu().detach().numpy(),
            "idio": pos_neg
        }
    
    return {
        "image": tensor2im(out.squeeze()), 
        "latents": to_decode.cpu().detach().numpy()
    }

In [36]:
rc_latents = load_rc_latents()

data = load_data()


data["worker_id"] = data["worker_id"].fillna(data["sona_id"])

for this_id in IDS_TO_EXCLUDE:
    if this_id: 
        data = data.loc[~data["worker_id"].str.contains(this_id, case=False, na=False)]

# 3. Check how many participants remain
print(f"Participants after exclusion: {len(data['worker_id'].dropna().unique())}")
print(f"Unique worker_ids: {data['worker_id'].unique()}")
data = clean_data(data) # get rid of weird subjects
meta_data = get_meta_data(data)
good_subject_ids = meta_data.loc[~meta_data["is_bad_subject"]]["worker_id"].tolist()
main_data_orig = get_main_data(data, include_repeat_data=True)
main_data = main_data_orig.loc[main_data_orig["worker_id"].isin(good_subject_ids)]
# keep only the main experiment phase
main_data = main_data.loc[main_data["experiment_phase"] == "main"]

Participants after exclusion: 3
Unique worker_ids: [nan 'TEST' '88626' '94144']


/tmp/ipykernel_1472/4266034515.py:81: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  rc_data['repeat'] = rc_data['repeat'].replace(


In [14]:
repeat_counts = data.groupby('worker_id')['repeat'].sum()
print("Repeat trials per participant:")
print(repeat_counts[['88626', '94144']])

Repeat trials per participant:
worker_id
88626    30
94144    30
Name: repeat, dtype: object


In [15]:
def analyze_trial_counts(data):

    rc_data = data[data['trial_type'] == 'single-stim-rev-cor-trial'].copy()

    rc_data['repeat'] = rc_data['repeat'].replace({'True': True, 'False': False, 'true': True, 'false': False})
    rc_data['repeat'] = rc_data['repeat'].fillna(False).astype(bool)
    
    trial_counts = rc_data.groupby(['worker_id', 'repeat']).size().unstack(fill_value=0)
    trial_counts = trial_counts.rename(columns={False: 'first_presentations', True: 'repeat_presentations'})
    
    # Count unique stimuli
    unique_stimuli = rc_data.groupby('worker_id')['stimulus_number'].nunique()
    
    # Combine results
    analysis_df = trial_counts.join(unique_stimuli.rename('unique_stimuli'))
    analysis_df['expected_pairs'] = 30  # Based on your experiment design
    
    # Calculate matching status
    analysis_df['complete_pairs'] = analysis_df[['first_presentations', 'repeat_presentations']].min(axis=1)
    analysis_df['missing_first'] = analysis_df['repeat_presentations'] - analysis_df['complete_pairs']
    analysis_df['missing_repeats'] = analysis_df['first_presentations'] - analysis_df['complete_pairs']
    
    return analysis_df

# Run analysis
trial_analysis = analyze_trial_counts(data)

# Display results for your participants
print("Complete Trial Analysis:")
print(trial_analysis.loc[['88626', '94144']])

# Detailed stimulus check for problematic cases
for worker_id in ['88626', '94144']:
    worker_data = data[data['worker_id'] == worker_id]
    print(f"\nParticipant {worker_id} stimulus counts:")
    print(worker_data['stimulus_number'].value_counts().head(40))

def calculate_proper_pairs(data):
    """Calculate properly matched pairs accounting for presentation order"""
    # Filter and clean data
    rc_data = data[data['trial_type'] == 'single-stim-rev-cor-trial'].copy()
    rc_data['repeat'] = rc_data['repeat'].astype(bool)
    
    # Response mapping
    response_map = {
        "GAD": 1, "no GAD": -1, "MDD": 1, "no MDD": -1,
        "PTSD": 1, "no PTSD": -1, "BPD": 1, "no BPD": -1,
        "yes": 1, "no": -1, "not sure": 0
    }
    rc_data['score'] = rc_data['response_label'].map(response_map)
    
    # Group by participant
    results = []
    
    for worker_id, worker_data in rc_data.groupby('worker_id'):
        print(f"\nProcessing participant {worker_id}")
        
        # Get all stimuli with exactly 2 presentations
        stimulus_counts = worker_data['stimulus_number'].value_counts()
        paired_stimuli = stimulus_counts[stimulus_counts == 2].index
        
        print(f"- Found {len(paired_stimuli)} properly paired stimuli")
        
        # Verify pairing (1 first, 1 repeat)
        valid_pairs = []
        for stim in paired_stimuli:
            stim_data = worker_data[worker_data['stimulus_number'] == stim]
            if len(stim_data[stim_data['repeat']]) == 1 and len(stim_data[~stim_data['repeat']]) == 1:
                first = stim_data[~stim_data['repeat']].iloc[0]['score']
                repeat = stim_data[stim_data['repeat']].iloc[0]['score']
                valid_pairs.append((first, repeat))
            else:
                print(f"- Bad pairing for stimulus {stim}:")
                print(f"  First presentations: {len(stim_data[~stim_data['repeat']])}")
                print(f"  Repeats: {len(stim_data[stim_data['repeat']])}")
        
        # Calculate reliability
        if len(valid_pairs) >= 2:
            first_resp, repeat_resp = zip(*valid_pairs)
            try:
                corr = pearsonr(first_resp, repeat_resp)[0]
                print(f"- Reliability: r = {corr:.3f} (based on {len(valid_pairs)} valid pairs)")
                results.append({
                    'worker_id': worker_id,
                    'condition': worker_data['condition'].iloc[0],
                    'reliability': corr,
                    'valid_pairs': len(valid_pairs),
                    'total_repeats': len(worker_data[worker_data['repeat']])
                })
            except:
                print("- Couldn't calculate reliability")
        else:
            print("- Insufficient valid pairs")
    
    return pd.DataFrame(results)

# Run analysis
pair_results = calculate_proper_pairs(data)
print("\nFinal Pairing Results:")
print(pair_results)

Complete Trial Analysis:
           first_presentations  repeat_presentations  unique_stimuli  \
worker_id                                                              
88626                      300                    30             300   
94144                      300                    30             300   

           expected_pairs  complete_pairs  missing_first  missing_repeats  
worker_id                                                                  
88626                  30              30              0              270  
94144                  30              30              0              270  

Participant 88626 stimulus counts:
stimulus_number
34.0     2
143.0    2
112.0    2
241.0    2
69.0     2
286.0    2
228.0    2
61.0     2
236.0    2
33.0     2
2.0      2
136.0    2
157.0    2
3.0      2
94.0     2
199.0    2
201.0    2
40.0     2
264.0    2
163.0    2
298.0    2
238.0    2
220.0    2
66.0     2
48.0     2
115.0    2
108.0    2
97.0     2
83.0     2
146.0    2


/tmp/ipykernel_1472/127943966.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  rc_data['repeat'] = rc_data['repeat'].replace({'True': True, 'False': False, 'true': True, 'false': False})


In [16]:
duplicates = data.duplicated(['worker_id', 'stimulus_number', 'repeat'], keep=False)
print(f"Duplicate trials: {len(data[duplicates])}")

Duplicate trials: 964


In [17]:
data.anon_id.unique()

array(['b7rgecm91q5gwpo', 'pe2mm4qzcy21xoo', '8ye2tdatja9shfz',
       '6eevj78xarytlk1', '5rj2owedhsefkas', 'chl1qgc2mjc03q0'],
      dtype=object)

In [18]:
data.worker_id.nunique()

3

In [19]:
data.to_csv("2-participant-test-data-sona/jspsych_data_new.csv")

In [20]:
meta_data.to_csv(SAVE_PATH / "2025-04-29_subject_meta_data_new.csv")

In [21]:
main_data.dtypes

rt                                       float64
url                                       object
experiment_phase                          object
refresh_count                              int64
trial_type                                object
trial_index                                int64
time_elapsed                               int64
experiment_name                           object
assignment_id                            float64
hit_id                                   float64
worker_id                                 object
turk_submit_to                           float64
preview_mode                              object
outside_turk                              object
platform                                  object
start_time                   datetime64[ns, UTC]
condition                                 object
end_time                     datetime64[ns, UTC]
total_time                                 int64
ip_address                                object
version_date        

In [22]:

# First ensure all required columns exist
if 'instructions_condition' not in data.columns:
    data['instructions_condition'] = data.get('condition', 'default')

# Clean and preprocess data
data = clean_data(data)
meta_data = get_meta_data(data)
good_subject_ids = meta_data.loc[~meta_data["is_bad_subject"]]["anon_id"].tolist()
main_data = get_main_data(data, include_repeat_data=True)
main_data = main_data.loc[main_data["anon_id"].isin(good_subject_ids)]
main_data = main_data.loc[main_data["experiment_phase"] == "main"]

# Create a results container DataFrame
results_df = pd.DataFrame(columns=[
    'anon_id', 
    'sona_id',
    'condition', 
    'positive_count',
    'negative_count',
    'neither_count',
    'status',
    'error_message'
])

# Get sona_id mapping
sona_mapping = main_data[['anon_id', 'sona_id']].drop_duplicates().set_index('anon_id')['sona_id']

# Process each subject
for anon_id, group in main_data.groupby("anon_id"):
    try:
        condition = group["condition"].unique()[0].lower()
        CATEGORY_LABEL_DICT = {
            "positive": condition.upper(),
            "negative": f"no {condition.upper()}",
            "neither": "not sure"
        }
        
        response_counts = group["response_label"].value_counts()
        
        result = get_results(
            group,
            latents=rc_latents,
            positive_category=CATEGORY_LABEL_DICT["positive"],
            negative_category=CATEGORY_LABEL_DICT["negative"],
            neither_category=CATEGORY_LABEL_DICT["neither"],
            desired_num_trials=10,
            num_random_latents=2
        )
        
        # Add to results DataFrame
        results_df.loc[len(results_df)] = {
            'anon_id': anon_id,
            'sona_id': sona_mapping.get(anon_id, 'N/A'),
            'condition': condition,
            'positive_count': response_counts.get(CATEGORY_LABEL_DICT["positive"], 0),
            'negative_count': response_counts.get(CATEGORY_LABEL_DICT["negative"], 0),
            'neither_count': response_counts.get(CATEGORY_LABEL_DICT["neither"], 0),
            'status': 'Success',
            'error_message': ''
        }
        
    except Exception as e:
        results_df.loc[len(results_df)] = {
            'anon_id': anon_id,
            'sona_id': sona_mapping.get(anon_id, 'N/A'),
            'condition': condition,
            'positive_count': 0,
            'negative_count': 0,
            'neither_count': 0,
            'status': 'Failed',
            'error_message': str(e)
        }

# Display first few rows
print("\nProcessing Results:")
display(results_df.head())

# Show summary statistics
print("\nSummary Statistics:")
display(results_df.groupby('condition').agg({
    'positive_count': ['mean', 'std'],
    'negative_count': ['mean', 'std'],
    'neither_count': ['mean', 'std'],
    'status': lambda x: (x == 'Success').mean()
}).rename(columns={'<lambda>': 'success_rate'}))

/tmp/ipykernel_1472/1503676477.py:81: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  rc_data['repeat'] = rc_data['repeat'].replace(


subject = 5rj2owedhsefkas
conditions: ['gad']
Positive latents: 35
Negative latents: 226
Neither latents: 39
subject = 6eevj78xarytlk1
conditions: ['mdd']
Replacing 'not sure' with random average...
Positive latents: 126
Negative latents: 174
Neither latents: 2
subject = 8ye2tdatja9shfz
conditions: ['gad']
Positive latents: 150
Negative latents: 148
Neither latents: 2
subject = b7rgecm91q5gwpo
conditions: ['gad']
Replacing 'not sure' with random average...
Positive latents: 107
Negative latents: 193
Neither latents: 2
subject = chl1qgc2mjc03q0
conditions: ['mdd']
Positive latents: 116
Negative latents: 150
Neither latents: 34
subject = pe2mm4qzcy21xoo
conditions: ['gad']
Replacing 'not sure' with random average...
Positive latents: 127
Negative latents: 173
Neither latents: 2

Processing Results:


,anon_id,sona_id,condition,positive_count,negative_count,neither_count,status,error_message
0,5rj2owedhsefkas,88626,gad,35,226,39,Success,
1,6eevj78xarytlk1,TEST,mdd,126,174,0,Success,
2,8ye2tdatja9shfz,NaN,gad,150,148,2,Success,
3,b7rgecm91q5gwpo,NaN,gad,107,193,0,Success,
4,chl1qgc2mjc03q0,94144,mdd,116,150,34,Success,



Summary Statistics:


positive_count            negative_count            neither_count  \
                    mean        std           mean        std          mean   
condition                                                                     
gad               104.75  49.708316          185.0  32.954514         10.25   
mdd               121.00   7.071068          162.0  16.970563         17.00   

                           status  
                 std success_rate  
condition                          
gad        19.189841          1.0  
mdd        24.041631          1.0

In [23]:
print("Main data shape:", main_data.shape)
print("Unique subjects:", main_data['anon_id'].nunique())
print("Condition distribution:\n", main_data['condition'].value_counts())

Main data shape: (1800, 52)
Unique subjects: 6
Condition distribution:
 condition
gad    1200
mdd     600
Name: count, dtype: int64


In [24]:
print("Original data shape:", data.shape)
print("Columns in raw data:", data.columns.tolist())
print("Unique conditions in raw data:", data['condition'].unique())

Original data shape: (2028, 50)
Columns in raw data: ['rt', 'url', 'experiment_phase', 'refresh_count', 'trial_type', 'trial_index', 'time_elapsed', 'experiment_name', 'assignment_id', 'hit_id', 'worker_id', 'turk_submit_to', 'preview_mode', 'outside_turk', 'platform', 'start_time', 'condition', 'end_time', 'total_time', 'ip_address', 'version_date', 'anon_id', 'debug_mode', 'completion_code', 'seed', 'redirect_url', 'worker_info', 'browser_events', 'success', 'screen_width', 'screen_height', 'window_width', 'window_height', 'form_name', 'form_id', 'instructions_viewed_count', 'view_history', 'stimulus', 'key_press', 'key_name', 'response_label', 'stimulus_number', 'attention_check_on', 'attention_check_passed', 'image_shown_count', 'repeat', 'responses', 'form_data', 'sona_id', 'instructions_condition']
Unique conditions in raw data: ['gad' 'mdd']


In [25]:
def debug_category_counts(subject_data, category_col="response_label"):
   

    response_counts = subject_data[category_col].value_counts()
    print("Total response counts:")
    print(response_counts)
    

    condition = subject_data["condition"].unique()[0]
    print(f"\nCondition: {condition}")
    
    expected_categories = {
        "positive": condition.upper(),
        "negative": f"no {condition.lower()}",
        "neither": "not sure"
    }
    
    for cat_type, cat_label in expected_categories.items():
        cat_data = subject_data[subject_data[category_col] == cat_label]
        print(f"\n{cat_type} category ('{cat_label}'):")
        print(f"- Count: {len(cat_data)}")
        
        if len(cat_data) > 0:
            print("- Sample stimulus indexes:", cat_data["stimulus_index"].head(5).tolist())
        else:
            print("Eh bokka levu")
    


for anon_id, group in main_data.groupby("anon_id"):
    print(f"\nDebugging subject: {anon_id}")
    debug_category_counts(group)


Debugging subject: 5rj2owedhsefkas
Total response counts:
response_label
no GAD      226
not sure     39
GAD          35
Name: count, dtype: int64

Condition: gad

positive category ('GAD'):
- Count: 35
- Sample stimulus indexes: [14, 1, 292, 103, 33]

negative category ('no gad'):
- Count: 0
Eh bokka levu

neither category ('not sure'):
- Count: 39
- Sample stimulus indexes: [298, 127, 185, 71, 209]

Debugging subject: 6eevj78xarytlk1
Total response counts:
response_label
no MDD    174
MDD       126
Name: count, dtype: int64

Condition: mdd

positive category ('MDD'):
- Count: 126
- Sample stimulus indexes: [224, 195, 27, 73, 62]

negative category ('no mdd'):
- Count: 0
Eh bokka levu

neither category ('not sure'):
- Count: 0
Eh bokka levu

Debugging subject: 8ye2tdatja9shfz
Total response counts:
response_label
GAD         150
no GAD      148
not sure      2
Name: count, dtype: int64

Condition: gad

positive category ('GAD'):
- Count: 150
- Sample stimulus indexes: [93, 170, 195, 

In [26]:
# See what response labels exist in the data
print(main_data['response_label'].unique())

# Check responses for one problematic participant
print(main_data[main_data['anon_id'] == '5rj2owedhsefkas']['response_label'].value_counts())

['no GAD' 'GAD' 'not sure' 'MDD' 'no MDD']
response_label
no GAD      226
not sure     39
GAD          35
Name: count, dtype: int64


In [27]:
grouped = main_data.groupby(by="anon_id")

results = {}
errors = []
error_outputs = []

checkpoint = False
save_output = True
for anon_id, group in grouped:
    condition = group['condition'].iloc[0].lower()  # e.g., "gad"
    category_map = get_condition_specific_labels(condition)  # Get labels for this condition
    
    try:
        results[anon_id] = get_results(
            group,
            latents=rc_latents,
            positive_category=category_map["positive"],  # "GAD"
            negative_category=category_map["negative"],  # "no GAD"
            neither_category=category_map["neither"],    # "not sure"
            save_output=save_output
        )
    except Exception as e:
        errors.append(anon_id)
        print(f"Error processing {anon_id} ({condition}): {str(e)}")

subject = 5rj2owedhsefkas
conditions: ['gad']
Positive latents: 35
Negative latents: 226
Neither latents: 39
subject = 6eevj78xarytlk1
conditions: ['mdd']
Replacing 'not sure' with random average...
Positive latents: 126
Negative latents: 174
Neither latents: 30
subject = 8ye2tdatja9shfz
conditions: ['gad']
8ye2tdatja9shfz: only 2 responses in 'not sure' category; adding more to reach 30...
Positive latents: 150
Negative latents: 148
Neither latents: 30
subject = b7rgecm91q5gwpo
conditions: ['gad']
Replacing 'not sure' with random average...
Positive latents: 107
Negative latents: 193
Neither latents: 30
subject = chl1qgc2mjc03q0
conditions: ['mdd']
Positive latents: 116
Negative latents: 150
Neither latents: 34
subject = pe2mm4qzcy21xoo
conditions: ['gad']
Replacing 'not sure' with random average...
Positive latents: 127
Negative latents: 173
Neither latents: 30


In [28]:
# Check if conditions are being mapped correctly
for anon_id in errors[:3]:  # First few error cases
    group = main_data[main_data['anon_id'] == anon_id]
    print(f"\nParticipant: {anon_id}")
    print("Condition:", group['condition'].unique())
    print("Response counts:")
    print(group['response_label'].value_counts())

In [29]:
errors

[]

In [30]:
error_outputs

[]

In [31]:
for s in errors:
    error_data = main_data.loc[main_data["anon_id"] == s]
    this_condition = error_data["condition"].unique()[0]
    # this_instruction = error_data["instructions_condition"].unique()[0]
    print(f"{s}: {this_condition}")
    display(error_data.response_label.value_counts())

In [30]:
df = pd.DataFrame.from_dict(results, orient="index")
df = df.reset_index(names="anon_id")

In [31]:
print(f"Total subjects: {main_data['anon_id'].nunique()}")
print(f"Columns: {main_data.columns.tolist()}")
print("\nSample response counts per subject:")
print(main_data.groupby('anon_id')['response_label'].value_counts().head(20))

Total subjects: 6
Columns: ['rt', 'url', 'experiment_phase', 'refresh_count', 'trial_type', 'trial_index', 'time_elapsed', 'experiment_name', 'assignment_id', 'hit_id', 'worker_id', 'turk_submit_to', 'preview_mode', 'outside_turk', 'platform', 'start_time', 'condition', 'end_time', 'total_time', 'ip_address', 'version_date', 'anon_id', 'debug_mode', 'completion_code', 'seed', 'redirect_url', 'worker_info', 'browser_events', 'success', 'screen_width', 'screen_height', 'window_width', 'window_height', 'form_name', 'form_id', 'instructions_viewed_count', 'view_history', 'stimulus', 'key_press', 'key_name', 'response_label', 'stimulus_number', 'attention_check_on', 'attention_check_passed', 'image_shown_count', 'repeat', 'responses', 'form_data', 'sona_id', 'instructions_condition', 'latent', 'stimulus_index']

Sample response counts per subject:
anon_id          response_label
5rj2owedhsefkas  no GAD            226
                 not sure           39
                 GAD               

In [32]:
print(f"Results dictionary length: {len(results)}")
print(f"Dictionary contents: {results}")
print(f"Total subjects in main_data: {main_data['anon_id'].nunique()}")
print("Sample subjects:", main_data['anon_id'].unique()[:5])

Results dictionary length: 6


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [33]:
print("Unique conditions in results dict:", {v['condition'] for v in results.values()})
print("Conditions in main_data:", main_data['condition'].value_counts())


Unique conditions in results dict: {'gad', 'mdd'}
Conditions in main_data: condition
gad    1200
mdd     600
Name: count, dtype: int64


In [34]:
df.head()

,anon_id,condition,positive,negative,neither,positive_mean,negative_mean,neither_mean,tmr_vector,all,...,tmr_r_CI95%_healthy,tmr_r_pval_healthy,tmr_r_BF10_healthy,tmr_r_power_healthy,tmr_r_n_SES,tmr_r_SES,tmr_r_CI95%_SES,tmr_r_pval_SES,tmr_r_BF10_SES,tmr_r_power_SES
0,5rj2owedhsefkas,gad,"[[-0.0033842002, -0.2568033, -0.013955138, -0....","[[-0.049884934, -0.13330416, 0.75021434, -0.01...","[[-0.16432706, -0.13880852, -0.14999004, -0.05...","[-0.022572309, -0.058678884, 0.06322245, -0.06...","[0.032278053, -0.058037408, 0.18282458, -0.005...","[-0.049581774, -0.10543307, 0.12157784, -0.043...","[-0.054850362, -0.00064147636, -0.11960213, -0...","[[-0.0033842002, -0.2568033, -0.013955138, -0....",...,"[-0.2, -0.03]",0.010012,1.511,0.732117,512,-0.138961,"[-0.22, -0.05]",1.621792e-03,7.839,0.884613
1,6eevj78xarytlk1,mdd,"[[-0.04632842, -0.097188525, 0.25203416, -0.13...","[[-0.13519934, -0.042867452, -0.39792407, -0.1...","[[-0.0686047, -0.07300926, -0.1085591, -0.1399...","[-0.006178745, -0.05480316, 0.099333666, -0.03...","[0.030745074, -0.071131654, 0.20549797, -0.004...","[0.05265927, -0.025983833, 0.19780299, -0.0086...","[-0.03692382, 0.016328495, -0.1061643, -0.0316...","[[-0.04632842, -0.097188525, 0.25203416, -0.13...",...,"[-0.2, -0.03]",0.008817,1.692,0.746350,512,-0.282712,"[-0.36, -0.2]",7.280091e-11,8.561e+07,0.999998
2,8ye2tdatja9shfz,gad,"[[-0.059184037, -0.0170536, -0.09908599, -0.10...","[[-0.07732462, -0.09022226, 0.43708304, -0.027...","[[-0.06835479, 0.08787264, 0.55617064, 0.00595...","[0.03816002, -0.07575642, 0.18199977, 0.001297...","[-0.0062656226, -0.053942673, 0.13479757, -0.0...","[0.006382062, -0.10676696, 0.12625815, -0.0074...","[0.04442564, -0.02181375, 0.0472022, 0.0391170...","[[-0.059184037, -0.0170536, -0.09908599, -0.10...",...,"[-0.03, 0.14]",0.220866,0.117,0.231860,512,-0.040129,"[-0.13, 0.05]",3.648457e-01,0.083,0.148128
3,b7rgecm91q5gwpo,gad,"[[-0.25735798, 0.2724218, -0.24117568, -0.1410...","[[-0.23275177, -0.15944773, 0.46859932, -0.108...","[[-0.06633769, -0.042682704, -0.12733231, -0.0...","[0.013812826, -0.041055657, 0.16309704, -0.039...","[0.016026681, -0.07714583, 0.15969585, -0.0047...","[0.044432685, 0.014727995, 0.17279267, 0.03358...","[-0.0022138553, 0.036090173, 0.00340119, -0.03...","[[-0.25735798, 0.2724218, -0.24117568, -0.1410...",...,"[-0.22, -0.05]",0.002453,5.367,0.858767,512,-0.120027,"[-0.2, -0.03]",6.546121e-03,2.208,0.777407
4,chl1qgc2mjc03q0,mdd,"[[-0.0014006937, -0.11714485, 0.81046593, -0.0...","[[-0.08235802, -0.06881271, -0.04816636, 0.058...","[[-0.08460752, 0.6828874, -0.11972974, -0.0103...","[0.030771745, -0.09429939, 0.17026405, -0.0301...","[0.0134354085, -0.07117092, 0.16436034, 0.0027...","[-0.029815054, 0.06859586, 0.11376471, -0.0619...","[0.017336337, -0.023128472, 0.005903706, -0.03...","[[-0.0014006937, -0.11714485, 0.81046593, -0.0...",...,"[-0.07, 0.1]",0.688933,0.06,0.068532,512,-0.002650,"[-0.09, 0.08]",9.523040e-01,0.055,0.050379


In [35]:
dataset = pd.merge(df, meta_data, on="anon_id", how="left", suffixes=("", "_meta"))
dataset.head()

,anon_id,condition,positive,negative,neither,positive_mean,negative_mean,neither_mean,tmr_vector,all,...,not_sure_rt_sd,screen_width,screen_height,screen_area,seriousness_low,screen_size_low,interrupted_survey,previously_participated,reliability_low,is_bad_subject
0,5rj2owedhsefkas,gad,"[[-0.0033842002, -0.2568033, -0.013955138, -0....","[[-0.049884934, -0.13330416, 0.75021434, -0.01...","[[-0.16432706, -0.13880852, -0.14999004, -0.05...","[-0.022572309, -0.058678884, 0.06322245, -0.06...","[0.032278053, -0.058037408, 0.18282458, -0.005...","[-0.049581774, -0.10543307, 0.12157784, -0.043...","[-0.054850362, -0.00064147636, -0.11960213, -0...","[[-0.0033842002, -0.2568033, -0.013955138, -0....",...,1454.684858,1470.0,956.0,1405320.0,False,False,False,False,False,False
1,6eevj78xarytlk1,mdd,"[[-0.04632842, -0.097188525, 0.25203416, -0.13...","[[-0.13519934, -0.042867452, -0.39792407, -0.1...","[[-0.0686047, -0.07300926, -0.1085591, -0.1399...","[-0.006178745, -0.05480316, 0.099333666, -0.03...","[0.030745074, -0.071131654, 0.20549797, -0.004...","[0.05265927, -0.025983833, 0.19780299, -0.0086...","[-0.03692382, 0.016328495, -0.1061643, -0.0316...","[[-0.04632842, -0.097188525, 0.25203416, -0.13...",...,NaN,1542.0,870.0,1341540.0,False,False,False,False,False,False
2,8ye2tdatja9shfz,gad,"[[-0.059184037, -0.0170536, -0.09908599, -0.10...","[[-0.07732462, -0.09022226, 0.43708304, -0.027...","[[-0.06835479, 0.08787264, 0.55617064, 0.00595...","[0.03816002, -0.07575642, 0.18199977, 0.001297...","[-0.0062656226, -0.053942673, 0.13479757, -0.0...","[0.006382062, -0.10676696, 0.12625815, -0.0074...","[0.04442564, -0.02181375, 0.0472022, 0.0391170...","[[-0.059184037, -0.0170536, -0.09908599, -0.10...",...,388.908730,1440.0,900.0,1296000.0,False,False,False,False,False,False
3,b7rgecm91q5gwpo,gad,"[[-0.25735798, 0.2724218, -0.24117568, -0.1410...","[[-0.23275177, -0.15944773, 0.46859932, -0.108...","[[-0.06633769, -0.042682704, -0.12733231, -0.0...","[0.013812826, -0.041055657, 0.16309704, -0.039...","[0.016026681, -0.07714583, 0.15969585, -0.0047...","[0.044432685, 0.014727995, 0.17279267, 0.03358...","[-0.0022138553, 0.036090173, 0.00340119, -0.03...","[[-0.25735798, 0.2724218, -0.24117568, -0.1410...",...,NaN,1440.0,900.0,1296000.0,False,False,False,False,False,False
4,chl1qgc2mjc03q0,mdd,"[[-0.0014006937, -0.11714485, 0.81046593, -0.0...","[[-0.08235802, -0.06881271, -0.04816636, 0.058...","[[-0.08460752, 0.6828874, -0.11972974, -0.0103...","[0.030771745, -0.09429939, 0.17026405, -0.0301...","[0.0134354085, -0.07117092, 0.16436034, 0.0027...","[-0.029815054, 0.06859586, 0.11376471, -0.0619...","[0.017336337, -0.023128472, 0.005903706, -0.03...","[[-0.0014006937, -0.11714485, 0.81046593, -0.0...",...,538.759560,1440.0,900.0,1296000.0,False,False,False,False,False,False


In [36]:
dataset.shape

(6, 320)

In [37]:
dataset.columns[:10]

Index(['anon_id', 'condition', 'positive', 'negative', 'neither',
       'positive_mean', 'negative_mean', 'neither_mean', 'tmr_vector', 'all'],
      dtype='object')

In [38]:
grouped_by_condition_latents = dataset.groupby(by="condition")[
    [
        "positive_mean",
        "negative_mean",
        "neither_mean",
        # XXX come back to scores
        # "left_right_score",        
    ]
]

rows = []

for condition, group in grouped_by_condition_latents:
    print(f"{condition=}, {group.shape[0]} participants")
    pos_group_mean = np.mean(group["positive_mean"].to_numpy())
    neg_group_mean = np.mean(group["negative_mean"].to_numpy())
    neither_group_mean = np.mean(group["neither_mean"].to_numpy())
    rows.append({
        "condition": condition,
        "positive_mean": pos_group_mean,
        "negative_mean": neg_group_mean,
        "neither_mean": neither_group_mean,
    })

condition_mean_representation_df = pd.DataFrame(rows)

condition='gad', 4 participants
condition='mdd', 2 participants


In [39]:
# all _usable_ subjects
grouped_by_condition_latents.size()

condition
gad    4
mdd    2
dtype: int64

In [40]:
# all subjects
main_data.drop_duplicates("anon_id").groupby(by=["condition"]).size()

condition
gad    4
mdd    2
dtype: int64

In [41]:
main_data.groupby(by=["condition"])["response_label"].value_counts()

condition  response_label
gad        no GAD            740
           GAD               419
           not sure           41
mdd        no MDD            324
           MDD               242
           not sure           34
Name: count, dtype: int64

In [42]:
for (condition), group in grouped_by_condition_latents:
    print(condition)

gad
mdd


In [43]:
condition_mean_representation_df.head()

,condition,positive_mean,negative_mean,neither_mean
0,gad,"[0.010733629, -0.06350418, 0.14510846, -0.0311...","[0.0146316085, -0.060734272, 0.1575002, -0.015...","[0.0032673862, -0.07547915, 0.14068487, -0.006..."
1,mdd,"[0.0122965, -0.07455128, 0.13479885, -0.032876...","[0.022090241, -0.07115129, 0.18492916, -0.0006...","[0.011422108, 0.021306012, 0.15578385, -0.0352..."


### Visualization of Prototypes

The visualization pipeline transformed latent vectors back into face images

In [44]:
def create_mental_representation_reel(
    mean_dict, decoder=ed, min_sd=-2, max_sd=2, step=0.5
):
    images = []
    result_latents = []
    for s in np.arange(min_sd, max_sd + step, step):
        result = create_mental_representations(
            encoder_decoder=decoder,
            positive=mean_dict["positive"],
            negative=mean_dict["negative"],
            neutral=mean_dict["neither"],
            step_num=s,
            norm=True,
        )
        images.append(result["image"])
        result_latents.append(result["latents"])

    return {
        "images": images,
        "latents": result_latents,
        "reel": get_concat_h_multi_resize(images),
    }

In [45]:
def save_average_face_reels_by_condition(df, min_sd=-2, max_sd=2, step=0.5):
    avg_images = {}
    for index, row in df.iterrows():
        condition = row["condition"]
        avg_images[condition] = create_mental_representation_reel(
            mean_dict={
                "positive": row["positive_mean"],
                "negative": row["negative_mean"],
                "neither": row["neither_mean"],
            },
            min_sd=min_sd,
            max_sd=max_sd,
            step=step,
        )

    
    average_save_path = SAVE_PATH / "averages"
    if not average_save_path.exists():
        average_save_path.mkdir(parents=True)

    for condition_name, v in avg_images.items():
        v["reel"].save(average_save_path / f"{condition_name}.png")        
        for s, img in zip(np.arange(min_sd, max_sd + step, step), v["images"]):
            img.save(average_save_path / f"{condition_name}_{s}.png")


def save_average_face_reels_by_condition_sex(df, min_sd=-2, max_sd=2, step=0.5):
    avg_images = {}
    for index, row in df.iterrows():
        condition = row["condition"]
        sex = row["sex"]
        condition_sex = f"{condition}-{sex}"
        avg_images[condition_sex] = create_mental_representation_reel(
            mean_dict={
                "positive": row["positive_mean"],
                "negative": row["negative_mean"],
                "neither": row["neither_mean"],
            },
            min_sd=min_sd,
            max_sd=max_sd,
            step=step,
        )

    
    average_save_path = SAVE_PATH / "averages"
    if not average_save_path.exists():
        average_save_path.mkdir(parents=True)

    for condition_name, v in avg_images.items():
        v["reel"].save(average_save_path / f"{condition_name}.png")        
        for s, img in zip(np.arange(min_sd, max_sd + step, step), v["images"]):
            img.save(average_save_path / f"{condition_name}_{s}.png")
            

            
            
def save_average_face_reels_by_condition_scenario(df, min_sd=-2, max_sd=2, step=0.5):
    avg_images = {}
    for index, row in df.iterrows():
        condition = row["condition"]
        scenario = row["scenario"]
        condition_scenario = f"{condition}-{scenario}"
        avg_images[condition_scenario] = create_mental_representation_reel(
            mean_dict={
                "positive": row["positive_mean"],
                "negative": row["negative_mean"],
                "neither": row["neither_mean"],
            },
            min_sd=min_sd,
            max_sd=max_sd,
            step=step,
        )

    
    average_save_path = SAVE_PATH / "averages"
    if not average_save_path.exists():
        average_save_path.mkdir(parents=True)

    for condition_name, v in avg_images.items():
        v["reel"].save(average_save_path / f"{condition_name}.png")        
        for s, img in zip(np.arange(min_sd, max_sd + step, step), v["images"]):
            img.save(average_save_path / f"{condition_name}_{s}.png")
            
            
def save_average_face_reels_by_sex_condition_scenario(df, min_sd=-2, max_sd=2, step=0.5):
    avg_images = {}
    for index, row in df.iterrows():
        sex = row["sex"]
        condition = row["condition"]
        scenario = row["scenario"]
        sex_condition_scenario = f"{sex}-{condition}-{scenario}"
        avg_images[sex_condition_scenario] = create_mental_representation_reel(
            mean_dict={
                "positive": row["positive_mean"],
                "negative": row["negative_mean"],
                "neither": row["neither_mean"],
            },
            min_sd=min_sd,
            max_sd=max_sd,
            step=step,
        )

    
    average_save_path = SAVE_PATH / "averages"
    if not average_save_path.exists():
        average_save_path.mkdir(parents=True)

    for condition_name, v in avg_images.items():
        v["reel"].save(average_save_path / f"{condition_name}.png")        
        for s, img in zip(np.arange(min_sd, max_sd + step, step), v["images"]):
            img.save(average_save_path / f"{condition_name}_{s}.png")

In [46]:
min_sd = -1.5
max_sd = 1.5
step = 0.5
save_average_face_reels_by_condition(df=condition_mean_representation_df, min_sd=min_sd, max_sd=max_sd, step=step)